# LM-Infinite: Zero-Shot Extreme Length Generalization for Large Language Models [1]

## Motiváció

A nagy nyelvi modellek általában rövid szövegrészleteken vannak betanítva, a Transformer architektúra négyzetes komplexitása miatt. Ennek következtében teljesítményük drasztikusan romlik, ha a bemenet hosszabb, mint amivel a tanítás során találkoztak.

Ez súlyosan korlátozza a gyakorlati alkalmazásokat, amelyek hosszú kontextust igényelnek, példaul:

- tudományos cikkek feldolgozása,
- forráskódok elemzése,
- hosszú párbeszédek kezelése.

A szerzők elemzéssel és emprikus vizsgálatokkal három fő tényezőt azonosítanak, amelyek a hosszú kontextusra való általánosítás sikertelenségét okozzák. A korábban használt technikák - például a figyelmi ablak csonkolása vagy a relatív pozíciós kódolás - nem alkalmazhatóak előretanított modellekre, finomhangolás nélkül.

Ezekre a kihíváokra válaszul a szerzők az LM-Infinite módszert javasolják, amely egyszerű és hatékony megoldást nyújt. A módszer paraméter-frissítés nélkül működik, és lehetővé teszi, hogy a 2000 vagy 4000 hosszúságu szakaszokon betanított LLM-ek akár 200 millió token hosszú bemenetekre is általánosítsanak, miközben megőrzik a perplexitásukat és a generálási minőségüket.

## Miért nem általánosítanak az LLM-ek hosszú kontextusra?

A szerzők három fő tényezőt azonosítottak, amelyek miatt a Transformer-alapú nyelvi modellek nem képesek a tanítási szekvenciahosszon túli bemeneteket megfelelően kezelni.

### Nem látott távolságok a tokenek között

A relatív pozíciós kódolást használó modellekben két token közötti figyelem súlya a relatív távolságuktól függ. Ha a szekvencia hosszabb, mint a tanítási időben látott maximális hossz, akkor olyan távolságértékek is megjelenhetnek, amelyekkel a modell so.sem találkozott.

Ez a probléma azért jelentkezik, mert a pozíciós beágyazások a tanítási tartományon kívüli értékekre való extrapolációja instabil lehet. A szerzők megfigyelték, hogy ezek a nem látott távolságok a figyelem logitjeinek "robbanását" okozhatják, ami a modell teljesítményének drasztikus romlásához vezet.

Például, ha egy modell legfeljebb 4096 token hosszú szekvenciákon volt tanítva, akkor a 4096-nál nagyobb relatív távolságok nem látható értékeknek számítanak, és a modell nem tudja, hogyan kezelje ezeket.

### Nem látott tokenszám

Hosszabb szekvenciák esetén a tokeneknek tübb más tokenre kell figyelmüket osztaniuk. A figyelem mechanizmus softmax normalizációja miatt, ha több token verseng a figyelemért, az egyes tokenekre jutó figyelemsúlyok kisebbek lesznek, és az eloszlás "szétterül".

A probléma az, hogy a modell tanítása során sosem látott ennyi együtthatót a softmac normalizációban. A szerzők megfigyelték, hogy ez a figyelem entrópiájának a tanítási tartományon túli növekedést okozhatja - azaz a figyelmi eloszlás túl egyenletessé váluk, és a modell nehezen különbözteti meg a fontos és kevésbé fontos tokeneket.

Ez különösen a későbbi pozíciókban lévő tokeneket érinti, amelyeknek a teljes hosszú kontextusra kell figyelnüik.

### A kezdeti tokenek különleges szerepe

A szerzők megfigyelték, hogy a szekvencia elején lévő tokenek más statisztikai helyzetben vannak, mint a többi token:

- Mindig jelen vannak, bármilyen hosszú is a szekvencia.
- Minden későbbi token számára ezek a legmesszebb lévő tokenek.
- A figurájuk a teljes kontextusban állandó, míg a többi token pozíciója változik.

Ezért a kezdeti tokenek a figyelem szempontjából egyfajta "horgonyként" működnek. Ha ezeket a tokeneket eldobjuk - például egy sliding window megközelítésben - a teljesítmény jelentősen romlik.

A szerzők ezt empirikusan is igazolták: amikor a kezdeti tokeneket kizárják a figyelemből, a modell perplexitása drasztikusan megnő, még akkor is, ha egyébként elegendő lokális kontextust adunk neki.

## Hivatkozások

1. Chi Han, Qifan Wang, Hao Peng, Wenhan Xiong, Yu Chen, Heng Ji, Sinong Wang. LM-Infinite: Zero-Shot Extreme Length Generalization for Large Language Models. June 2024. https://doi.org/10.48550/arXiv.2308.16137